<a href="https://colab.research.google.com/github/wonkwonlee/likelion-k-digital-training-AI/blob/main/Artificial-Intelligence/nlp-cnn-practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

# csv 챗봇 데이터 로드
train_file = "/content/drive/MyDrive/Colab Notebooks/data_nlp/chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# features의 각 문장 별로 단어 시퀀스를 생성하여 코퍼스에 저장
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features] 
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
# 단어를 시퀀스 번호로 변환
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index
# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN = 15 
# 빈 공간을 0으로 채우는 패딩 작업
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [ ]:
# 학습용, 검증용, 테스트용 데이터셋 생성 (7:2:1)
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels)) 
ds = ds.shuffle(len(features))  # 데이터 셔플

train_size = int(len(padded_seqs) * 0.7)  # 학습용 데이터
val_size = int(len(padded_seqs) * 0.2)    # 검증용 데이터
test_size = int(len(padded_seqs) * 0.1)   # 테스트용 데이터

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼파라미터 기본값 설정 (상황에 따라 변화 가능) 
dropout_prob = 0.3  # 드롭아웃 확률
EMB_SIZE = 128  # 임베딩 사이즈
EPOCH = 10 # 에폭값
VOCAB_SIZE = len(word_index) + 1 # 전체 단어 수

# CNN 모델 정의 
# 입력 레이어 (단어 시퀀스 벡터 크기)
input_layer = Input(shape=(MAX_SEQ_LEN,))
# 임베딩 레이어
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer) 
# 드롭아웃 레이어 (기본 드롭아웃 확률은 0.5) - 과적합 대비
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

# 컨볼루션 레이어 1
conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1) # 풀링 레이어 1
# 컨볼루션 레이어 2
conv2 = Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb) 
pool2 = GlobalMaxPool1D()(conv2) # 풀링 레이어 2
# 컨볼루션 레이어 3
conv3 = Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb) 
pool3 = GlobalMaxPool1D()(conv3) # 풀링 레이어 3

# 병렬 처리된 feature map 결합
concat = concatenate([pool1, pool2, pool3])

# 완전 연결 레이어 (relu 활성화 함수)
hidden = Dense(128, activation=tf.nn.relu)(concat) 
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
# 출력 노드: 3가지 감정 클래스 
logits = Dense(3, name='logits')(dropout_hidden) 
predictions = Dense(3, activation=tf.nn.softmax)(logits)

# 모델 생성 
model = Model(inputs=input_layer, outputs=predictions)
# ADAM 옵티마이저 사용
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)


Epoch 1/10
414/414 [==============================] - 9s 20ms/step - loss: 0.9332 - accuracy: 0.5292 - val_loss: 0.4297 - val_accuracy: 0.8549
Epoch 2/10
414/414 [==============================] - 8s 19ms/step - loss: 0.4046 - accuracy: 0.8538 - val_loss: 0.2085 - val_accuracy: 0.9349
Epoch 3/10
414/414 [==============================] - 8s 19ms/step - loss: 0.2043 - accuracy: 0.9328 - val_loss: 0.1294 - val_accuracy: 0.9607
Epoch 4/10
414/414 [==============================] - 8s 19ms/step - loss: 0.1287 - accuracy: 0.9599 - val_loss: 0.0609 - val_accuracy: 0.9835
Epoch 5/10
414/414 [==============================] - 8s 18ms/step - loss: 0.0868 - accuracy: 0.9735 - val_loss: 0.0423 - val_accuracy: 0.9869
Epoch 6/10
414/414 [==============================] - 8s 18ms/step - loss: 0.0565 - accuracy: 0.9806 - val_loss: 0.0471 - val_accuracy: 0.9835
Epoch 7/10
414/414 [==============================] - 7s 18ms/step - loss: 0.0517 - accuracy: 0.9830 - val_loss: 0.0283 - val_accuracy: 0.9903

In [ ]:
# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1) 
print('Accuracy: %f' % (accuracy * 100)) 
print('loss: %f' % (loss))

60/60 [==============================] - 0s 3ms/step - loss: 0.0158 - accuracy: 0.9924
Accuracy: 99.238580
loss: 0.015790
